In [1]:
!apt-get install -y coinor-cbc
!pip install pyomo


'apt-get' is not recognized as an internal or external command,
operable program or batch file.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\MarianaLozano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
import pandas as pd
from pyomo.environ import *

# === 1. Cargar datos ===
clients_df = pd.read_csv('Datos/clients.csv')
depots_df = pd.read_csv('Datos/depots.csv')
vehicles_df = pd.read_csv('Datos/vehicles.csv')

# Conjuntos
D = ['MUN' + str(int(i)).zfill(2) for i in clients_df['LocationID']]
P = ['PTO' + str(int(i)).zfill(2) for i in depots_df['LocationID']]
V_set = list(vehicles_df['VehicleID'].astype(str))
N = P + D

# === Coordenadas ===
coords = {}

# Depot (ya está bien tu CSV ahora 💪)
for _, row in depots_df.iterrows():
    key = 'PTO' + str(int(row['LocationID'])).zfill(2)
    lat = float(row['Latitude'])
    lon = float(row['Longitude'])
    coords[key] = (lat, lon)
    print(f"[DEPOT] {key}: ({lat}, {lon})")

# Clientes
for _, row in clients_df.iterrows():
    key = 'MUN' + str(int(row['LocationID'])).zfill(2)
    lat = float(row['Latitude'])
    lon = float(row['Longitude'])
    coords[key] = (lat, lon)
    print(f"[CLIENT] {key}: ({lat}, {lon})")


# === 2. Calcular distancias (Haversine) ===
from math import radians, sin, cos, sqrt, atan2

def haversine(coord1, coord2):
    R = 6371  # km
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

dist = {}
for i in N:
    for j in N:
        if i != j:
            dist[(i, j)] = haversine(coords[i], coords[j])
            print(f"Distancia {i} -> {j}: {dist[(i, j)]:.2f} km")

# === Parámetros ===
Ft = 5000    # Tarifa flete COP/km
Cm = 700     # Mantenimiento COP/km

# Capacidad y autonomía
Q = dict(zip(vehicles_df['VehicleID'].astype(str), vehicles_df['Capacity']))
Autonomy = dict(zip(vehicles_df['VehicleID'].astype(str), vehicles_df['Range']))

# Demanda
demand = {}
for _, row in clients_df.iterrows():
    key = 'MUN' + str(int(row['LocationID'])).zfill(2)
    demand[key] = row['Demand']


# === 3. Modelo Pyomo ===
model = ConcreteModel()

# Sets
model.V = Set(initialize=V_set)
model.N = Set(initialize=N)
model.D = Set(initialize=D)

# Variables
model.x = Var(model.V, model.N, model.N, domain=Binary)
model.q = Var(model.V, model.D, domain=NonNegativeReals)

# === 4. Función objetivo ===
def total_cost_rule(m):
    return sum(
        (Ft * dist[i, j] + Cm * dist[i, j]) * m.x[v, i, j]
        for v in m.V for i in m.N for j in m.N if i != j
    )
model.TotalCost = Objective(rule=total_cost_rule, sense=minimize)

# === 5. Restricciones ===

# (R1) Cada cliente es visitado una vez
def one_visit_per_client_rule(m, j):
    return sum(m.x[v, i, j] for v in m.V for i in m.N if i != j) == 1
model.VisitOnce = Constraint(model.D, rule=one_visit_per_client_rule)

# (R2) Conservación de flujo
def flow_conservation_rule(m, v, k):
    return sum(m.x[v, i, k] for i in m.N if i != k) == sum(m.x[v, k, j] for j in m.N if j != k)
model.FlowConservation = Constraint(model.V, model.N, rule=flow_conservation_rule)

# (R3) Capacidad de vehículo
def capacity_rule(m, v):
    return sum(m.q[v, j] for j in m.D) <= Q[v]
model.Capacity = Constraint(model.V, rule=capacity_rule)

# (R4) Satisfacer la demanda si visita
def demand_delivery_rule(m, v, j):
    return m.q[v, j] <= demand[j] * sum(m.x[v, i, j] for i in m.N if i != j)
model.DemandDelivery = Constraint(model.V, model.D, rule=demand_delivery_rule)

# (R5) Autonomía del vehículo (opcional pero recomendado)
def autonomy_rule(m, v):
    return sum(dist[i, j] * m.x[v, i, j] for i in m.N for j in m.N if i != j) <= Autonomy[v]
model.Autonomy = Constraint(model.V, rule=autonomy_rule)

# (R6) Los vehículos deben salir del depósito
def start_from_depot_rule(m, v):
    return sum(m.x[v, i, j] for i in P for j in m.N if i != j) == 1
model.StartFromDepot = Constraint(model.V, rule=start_from_depot_rule)

# (R7) Los vehículos deben regresar al depósito
def return_to_depot_rule(m, v):
    return sum(m.x[v, i, j] for i in m.N for j in P if i != j) == 1
model.ReturnToDepot = Constraint(model.V, rule=return_to_depot_rule)

# === Resolver ===
solver = SolverFactory('glpk', executable='glpk-4.65\\w64\\glpsol.exe')
results = solver.solve(model, tee=True)

# === Mostrar resultados ===
if results.solver.status == SolverStatus.ok:
    print("\n¡Solución encontrada!")
    total_cost = model.TotalCost()
    print(f"Costo total: ${total_cost:.2f} COP")
    
    for v in model.V:
        route = []
        current = None
        
        # Encuentra el nodo de inicio (depósito)
        for i in P:
            for j in model.N:
                if i != j and value(model.x[v, i, j]) > 0.5:
                    route.append(i)
                    current = j
                    break
            if current:
                break
        
        # Si no hay ruta para este vehículo, continuar
        if not current:
            continue
            
        # Recorrer la ruta
        while current not in P:
            route.append(current)
            for j in model.N:
                if current != j and value(model.x[v, current, j]) > 0.5:
                    current = j
                    break
        
        route.append(current)  # Añadir el retorno al depósito
        
        # Calcular distancia y carga
        total_distance = sum(dist[route[i], route[i+1]] for i in range(len(route)-1))
        total_load = sum(demand.get(node, 0) for node in route if node in D)
        
        print(f"\nRuta del vehículo {v}:")
        print(" -> ".join(route))
        print(f"Distancia total: {total_distance:.2f} km")
        print(f"Carga total: {total_load} unidades")
        
        # Mostrar entregas detalladas
        print("Detalle de entregas:")
        for node in route:
            if node in D:
                delivered = value(model.q[v, node])
                if delivered > 0:
                    print(f"  {node}: {delivered} unidades")
else:
    print("No se encontró una solución factible.")

[DEPOT] PTO01: (4.743359, -74.153536)
[CLIENT] MUN02: (4.59795431125545, -74.09893796560621)
[CLIENT] MUN03: (4.687820646838871, -74.07557103763986)
[CLIENT] MUN04: (4.70949446000624, -74.10708524062704)
[CLIENT] MUN05: (4.605029068682624, -74.09727965657427)
[CLIENT] MUN06: (4.648463876533332, -74.16464148202755)
[CLIENT] MUN07: (4.662137416953968, -74.12083799988112)
[CLIENT] MUN08: (4.697499030379109, -74.02213076607309)
[CLIENT] MUN09: (4.649416884236942, -74.17207549744595)
[CLIENT] MUN10: (4.606310650273935, -74.15615257246444)
[CLIENT] MUN11: (4.557379705282216, -74.09041145358674)
[CLIENT] MUN12: (4.591594072172954, -74.17802255204528)
[CLIENT] MUN13: (4.7564172406324055, -74.1015410917749)
[CLIENT] MUN14: (4.646217006050524, -74.09690889182339)
[CLIENT] MUN15: (4.725912125314368, -74.1219200708342)
[CLIENT] MUN16: (4.604168478560718, -74.0942948461378)
[CLIENT] MUN17: (4.557320898243896, -74.11138839002187)
[CLIENT] MUN18: (4.615869066082658, -74.12463941285208)
[CLIENT] MUN19